In [1]:
#get the latest set of images from UCAR
#Latest can be found at the catalog file
#cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/satellite/SFC-T/SUPER-NATIONAL_1km/current/catalog.xml')

In [2]:
#%%python3
# Magic not working

In [5]:
#link to the latest satellite imagery
latest <-'http://thredds.ucar.edu/thredds/dodsC/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170825_1900.gini'


In [6]:
latest

[1] "http://thredds.ucar.edu/thredds/dodsC/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170825_1900.gini"

In [7]:
#read the GINI as netCDF file
# load the ncdf4 package
library(ncdf4)

In [8]:
# open a netCDF file
ncin <- nc_open(latest)
print(ncin)

File http://thredds.ucar.edu/thredds/dodsC/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170825_1900.gini (NC_FORMAT_CLASSIC):

     2 variables (excluding dimension variables):
        char Stereographic[maxStrlen64]   
            grid_mapping_name: stereographic
            longitude_of_projection_origin: -105
            latitude_of_projection_origin: 90
            scale_factor_at_projection_origin: 0.93301269409307
            earth_radius: 6371229
            _CoordinateTransformType: Projection
            _CoordinateAxes: x y 
        byte SFC_T[x,y,time]   
            _Unsigned: true
            long_name: Surface Skin Temperature
            units: N/A
            scale_factor: 1
            add_offset: 0
            _CoordinateAxes: x y time

     4 dimensions:
        maxStrlen64  Size:64
        time  Size:1
            long_name: time since base date
            _CoordinateAxisType: Time
            units: msecs since 1970-01-01T00:00:00Z
       

In [25]:
# get proj x
x <- ncvar_get(ncin,"x")
nx <- dim(x)
head(nx)

[1] 1536

In [26]:
# get proj y
y <- ncvar_get(ncin,"y")
ny <- dim(y)
head(ny)

[1] 1008

In [27]:
print(c(nx,ny))

[1] 1536 1008


In [28]:
# get time
time <- ncvar_get(ncin,"time")
time

[1] 1.503688e+12

In [29]:
tunits <- ncatt_get(ncin,"time","units")
nt <- dim(time)
nt

[1] 1

In [30]:
dname <- "SFC_T"  # 
# get surface temperature
tmp_array <- ncvar_get(ncin,dname)
dlname <- ncatt_get(ncin,dname,"long_name")
dunits <- ncatt_get(ncin,dname,"units")
fillvalue <- ncatt_get(ncin,dname,"_FillValue")
dim(tmp_array)

[1] 1536 1008

In [31]:
library(rgdal)

xy <- as.data.frame(expand.grid(x,y))

Loading required package: sp
rgdal: version: 1.2-8, (SVN revision 663)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.11.3, released 2015/09/16
 Path to GDAL shared files: /usr/share/gdal/1.11
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-5 


In [32]:
coordinates(xy)= ~Var1 + Var2

In [33]:
proj.pol <- CRS('+proj=stere +lat_0=90 +lon_0=-105.0 +x_0=0.0 +y_0=0.0')
wgs <- CRS('+proj=longlat +datum=WGS84')
proj4string(xy) <- proj.pol
awgs <- spTransform(xy, wgs)
head(awgs)

SpatialPoints:
      Var1     Var2
1 168.5661 89.94532
2 168.5708 89.94539
3 168.5754 89.94546
4 168.5801 89.94553
5 168.5848 89.94560
6 168.5894 89.94567
Coordinate Reference System (CRS) arguments: +proj=longlat +datum=WGS84
+ellps=WGS84 +towgs84=0,0,0 

In [34]:
summary(awgs)

Object of class SpatialPoints
Coordinates:
            min      max
Var1 -179.99987 179.9999
Var2   89.90718  89.9966
Is projected: FALSE 
proj4string :
[+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0]
Number of points: 1548288

In [35]:
# quick ma
library(ggmap)    # loads ggplot2 as well
map <- get_map(location=rowMeans(bbox(awgs)), zoom=13)   # get Google map

Loading required package: ggplot2
Map from URL : http://maps.googleapis.com/maps/api/staticmap?center=89.951888,1.9e-05&zoom=13&size=640x640&scale=2&maptype=terrain&language=en-EN&sensor=false


In [36]:
str(awgs)

Formal class 'SpatialPoints' [package "sp"] with 3 slots
  ..@ coords     : num [1:1548288, 1:2] 169 169 169 169 169 ...
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:1548288] "1" "2" "3" "4" ...
  .. .. ..$ : chr [1:2] "Var1" "Var2"
  ..@ bbox       : num [1:2, 1:2] -180 89.9 180 90
  .. ..- attr(*, "dimnames")=List of 2
  .. .. ..$ : chr [1:2] "Var1" "Var2"
  .. .. ..$ : chr [1:2] "min" "max"
  ..@ proj4string:Formal class 'CRS' [package "sp"] with 1 slot
  .. .. ..@ projargs: chr "+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"


In [37]:
awgs[1548280]@coords[2]

[1] 89.90751

In [38]:
tmp_array[1,200]

[1] 0

In [39]:
str(tmp_array)

 int [1:1536, 1:1008] 0 0 0 0 0 0 0 0 0 0 ...


In [40]:
which(tmp_array!=0,arr.ind = T)


row,col
245,209
240,210
244,210
245,210
248,210
239,211
240,211
243,211
244,211
245,211


In [41]:
tmp_array[696,1000]

[1] 0

In [3]:
#library(chron)
library(RColorBrewer)
library(lattice)
library(ncdf4)
library(raster)


Loading required package: sp


In [14]:
rf='http://thredds.ucar.edu/thredds/ncss/satellite/SFC-T/SUPER-NATIONAL_1km/current/SUPER-NATIONAL_1km_SFC-T_20170825_1900.gini'
r <- raster("/home/ubuntu/SUPER-NATIONAL_1km_SFC-T_20170825_1900.gini",  varname = "SFC_T")

projection(r) <- "+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"
writeRaster(r, "Tair.grd", format='raster', overwrite=TRUE)
plot(r)

ERROR: Error in .rasterObjectFromFile(x, band = band, objecttype = "RasterLayer", : Cannot create a RasterLayer object from this file.


In [ ]:
#dfile="/Users/ajijohn/git/ebm/past_1981_Tsurface.nc"
dfile = "/Users/ajijohn/git/microclimRapi/vignettes/Tair_output_shade-0_height-6_interval-hourly_aggregation-avg_times-19810101-19991231_created-2017-05-18-0026.nc"

r <- raster(dfile,  varname = "SFC_T")

projection(r) <- "+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0"
writeRaster(r, "Tair.grd", format='raster', overwrite=TRUE)
plot(r)
